# 3) Generate the Output Metrics #

This notebook takes the metrics saved by notebook 1 and runs some additional post processing to generate 1 metric per grid cell per ensemble member <br>
<br>
Inputs: <br>
Metrics.nc - contains select output variables used for analysis <br>
growing_season_mask.nc - a helper dataset for masking out non growing season months
primaryPFT.nc - a helper dataset for selecting the most abundant pft (based on cveg) in each grid cell

Ouputs:
Metrics_GC.nc
Metrics_PFT.nc

## Load Libraries ##

In [56]:
import os
import warnings
import numpy as np
import pandas as pd
import xarray as xr
from tqdm import tqdm

repo_dir = os.getcwd()

## Load Data ##

In [19]:
Metrics = xr.open_dataset(repo_dir+'/input/Metrics.nc').drop_sel(pft = 'c3_crop')
growing_season_mask = xr.open_dataset(repo_dir+'/utils/GrowingSeasonMask.nc').mask.drop_sel(pft = 'c3_crop')
prim_pft=xr.open_dataset(repo_dir+'/utils/primaryPFT.nc')

## Calculate Growing Season GPP, T, and Cveg ##

In [22]:
B_growing_season = Metrics.BTRANMN.where(growing_season_mask)
B_growing_season_annual_mean = B_growing_season.groupby("time.year").mean(dim="time")

TOTVEGC_growing_season = Metrics.TOTVEGC.where(growing_season_mask)
TOTVEGC_growing_season_annual_mean = TOTVEGC_growing_season.groupby("time.year").mean(dim="time")

GPP_growing_season = Metrics.GPP.where(growing_season_mask)
GPP_growing_season_annual_mean = GPP_growing_season.groupby("time.year").mean(dim="time")

T_growing_season = Metrics.FCTR.where(growing_season_mask)
T_growing_season_annual_mean = T_growing_season.groupby("time.year").mean(dim="time")

B_growing_season_GC = (B_growing_season* TOTVEGC_growing_season).sum(dim='pft') / TOTVEGC_growing_season.sum(dim='pft')
T_growing_season_GC = T_growing_season.sum(dim = 'pft')
GPP_growing_season_GC = GPP_growing_season.sum(dim = 'pft')

B_growing_season_annual_mean_GC = (B_growing_season_annual_mean* TOTVEGC_growing_season_annual_mean).sum(dim='pft') / TOTVEGC_growing_season_annual_mean.sum(dim='pft')
GPP_growing_season_annual_mean_GC = GPP_growing_season_annual_mean.sum(dim = 'pft')
T_growing_season_annual_mean_GC = T_growing_season_annual_mean.sum(dim = 'pft')

## Calculate Drought Sensitivity ##

In [34]:
#PFT-level

import xarray as xr
import numpy as np
from tqdm import tqdm  # Progress bar

# Retrieve the coordinate for 'ens' (used for the fallback nan array)
ens_coord = GPP_growing_season_annual_mean.coords['ens']
ens_size = ens_coord.size

# List to collect our individual DataArrays
results = []

# Loop through every gridcell and pft with a progress bar on gridcell
for gi in tqdm(GPP_growing_season_annual_mean.coords['gridcell'].values, desc="Processing gridcells"):
    for pf in GPP_growing_season_annual_mean.coords['pft'].values:
        try:
            sub = B_growing_season_annual_mean.sel(gridcell = gi, pft = pf)
            minB_year1 = sub.mean(dim = 'ens').idxmin(dim = 'year').values.item()
            minB_year2 = sub.mean(dim = 'ens').drop_sel(year = minB_year1).idxmin(dim = 'year').values.item()
            minB_year3 = sub.mean(dim = 'ens').drop_sel(year = [minB_year1, minB_year2]).idxmin(dim = 'year').values.item()
            # Try to select the GPP slice for the given year.
            data_slice = GPP_growing_season_annual_mean.sel(gridcell=gi, pft=pf).sel(year=[minB_year1,minB_year2,minB_year3]).mean(dim = 'year')
        except Exception as e:
            # If an error occurs (e.g., year not found), assign a fallback DataArray of NaNs.
            data_slice = xr.DataArray(np.full((ens_size,), np.nan), dims=['ens'], coords={'ens': ens_coord}).rename('GPP')
        # Expand dimensions to include gridcell and pft in the resulting DataArray.
        data_slice = data_slice.expand_dims({'gridcell': [gi], 'pft': [pf]})
        results.append(data_slice)

# Combine the individual slices along the new 'gridcell' and 'pft' dimensions.
minyear_GPP_all = xr.combine_by_coords(results)

Processing gridcells: 100%|██████████| 400/400 [00:13<00:00, 30.15it/s]


In [36]:
#Gridcell-level

import xarray as xr
import numpy as np

# Retrieve the coordinate for 'ens' (used for the fallback nan array)
ens_coord = GPP_growing_season_annual_mean_GC.coords['ens']
ens_size = ens_coord.size

# List to collect our individual DataArrays
results = []

# Loop through every gridcell and pft with a progress bar on gridcell
for gi in tqdm(GPP_growing_season_annual_mean_GC.coords['gridcell'].values, desc="Processing gridcells"):
    try:
        sub = B_growing_season_annual_mean_GC.sel(gridcell = gi)
        minB_year1 = sub.mean(dim = 'ens').idxmin(dim = 'year').values.item()
        minB_year2 = sub.mean(dim = 'ens').drop_sel(year = minB_year1).idxmin(dim = 'year').values.item()
        minB_year3 = sub.mean(dim = 'ens').drop_sel(year = [minB_year1, minB_year2]).idxmin(dim = 'year').values.item()
        # Try to select the GPP slice for the given year.
        data_slice = GPP_growing_season_annual_mean_GC.sel(gridcell=gi).sel(year=[minB_year1,minB_year2,minB_year3]).mean(dim = 'year')
    except Exception as e:
        # If an error occurs (e.g., year not found), assign a fallback DataArray of NaNs.
        data_slice = xr.DataArray(np.full((ens_size,), np.nan), dims=['ens'], coords={'ens': ens_coord}).rename('GPP')
    
    # Expand dimensions to include gridcell and pft in the resulting DataArray.
    data_slice = data_slice.expand_dims({'gridcell': [gi]})
    results.append(data_slice)

# Combine the individual slices along the new 'gridcell' and 'pft' dimensions.
minyear_GPP_all_GC = xr.combine_by_coords(results)


Processing gridcells: 100%|██████████| 400/400 [00:01<00:00, 309.98it/s]


In [38]:
DroughtSens3Years_GC = (minyear_GPP_all_GC - GPP_growing_season_annual_mean_GC.mean(dim = 'year')) / GPP_growing_season_annual_mean_GC.mean(dim = 'year')
DroughtSens3Years = (minyear_GPP_all - GPP_growing_season_annual_mean.mean(dim = 'year')) / GPP_growing_season_annual_mean.mean(dim = 'year')

In [58]:
Metrics_PFT = xr.merge([GPP_growing_season.mean(dim = 'time'), B_growing_season.mean(dim = 'time').rename('B'), T_growing_season.mean(dim = 'time').rename('T'), DroughtSens3Years.rename({'GPP':'DroughtSens'})])
Metrics_PFT = Metrics_PFT.astype('float64')
Metrics_GC = xr.merge([GPP_growing_season_GC.mean(dim = 'time'), B_growing_season_GC.mean(dim = 'time').rename('B'), T_growing_season_GC.mean(dim = 'time').rename('T'), DroughtSens3Years_GC.rename({'GPP':'DroughtSens'})])
Metrics_GC = Metrics_GC.astype('float64')

In [61]:
Metrics_PFT.to_netcdf(repo_dir+'/input/Metrics_PFT.nc')
Metrics_GC.to_netcdf(repo_dir+'/input/Metrics_GC.nc')